In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats
from scipy.stats import shapiro
from scipy.stats.mstats import winsorize
from sklearn.linear_model import Lasso, Ridge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
df = pd.read_csv('./dataset/label지정.csv', encoding='CP949')                        ## 성휘 파일 경로
# df = pd.read_excel('../eeez/전처리파일/label지정.xlsx', index_col=0)                ## 재인 파일 경로
# df = pd.read_csv('../데이터자료/label지정.csv', encoding='CP949')                   ## 의영 파일 경로
# df = pd.read_excel('./csv/dateset2_label최종.xlsx', index_col=0)                   ## 경록 파일 경로
df

## Winsorizing

In [4]:
df_1 = df.copy()
df_1.drop(['회사명', '거래소코드', '회계년도_x', '상장일', '상장폐지일', '산업명', '소속코드','year', 'month'],axis=1,inplace=True)
for col in df_1.columns:
    df_1[col] = winsorize(df_1[col], limits= 0.01)

# Ridge

In [45]:

feature = df_1.drop(columns='label')
target = df_1['label']

#릿지 클래서
ridge = Ridge(alpha = 10)

neg_mse_scores = cross_val_score(ridge, feature, target, scoring="neg_mean_squared_error", cv=5)
rmse_scores = np.sqrt(-1 * neg_mse_scores)
avg_rmse = np.mean(rmse_scores)
print(neg_mse_scores,rmse_scores,avg_rmse)

ridge_alphas =[1000,2000,3000,4000,5000]
for alpha in ridge_alphas :
    ridge = Ridge(alpha = alpha)
    neg_mse_scores = cross_val_score(ridge, feature, target, scoring="neg_mean_squared_error", cv=5)
    avg_rmse = np.mean(np.sqrt(-1*neg_mse_scores))
    print('alpha {0} 일 떄 5folds 의 평균 RMSE : {1:.10f}' .format(alpha,avg_rmse))

# fig, axs = plt.subplots(figsize = (18,6), nrows = 1, ncols=5)

coeff_df = pd.DataFrame()

for pos, alpha in enumerate(ridge_alphas):
    ridge = Ridge(alpha= alpha)
    ridge.fit(feature,target)
    coeff = pd.Series(data=ridge.coef_, index=feature.columns)
    colname = 'alpha:' + str(alpha)
    coeff_df[colname] = coeff

    coeff = coeff.sort_values(ascending=False)
#     axs[pos].set_title(colname)
#     axs[pos].set_xlim(-3,6)
#     sns.barplot(x=coeff.values, y=coeff.index, ax =axs[pos])

# plt.show()


[-0.01082428 -0.01583049 -0.01147362 -0.01493988 -0.01184755] [0.10403978 0.12581929 0.10711499 0.12222879 0.10884643] 0.11360985508773096
alpha 1000 일 떄 5folds 의 평균 RMSE : 0.1136160039
alpha 2000 일 떄 5folds 의 평균 RMSE : 0.1136430915
alpha 3000 일 떄 5folds 의 평균 RMSE : 0.1136578420
alpha 4000 일 떄 5folds 의 평균 RMSE : 0.1136674002
alpha 5000 일 떄 5folds 의 평균 RMSE : 0.1136740411


## Alpha 값 조정하면서 최적의 Alpha값 찾기

In [46]:
ridge_alphas =[1000,2000,3000,4000,5000]
sort_column = 'alpha:'+str(ridge_alphas[0])
coeff_df.sort_values(by=sort_column, ascending=False)

,alpha:1000,alpha:2000,alpha:3000,alpha:4000,alpha:5000
[제조]순운전자본회전률,6.977861e-03,4.014931e-03,2.757899e-03,2.065798e-03,1.630485e-03
[제조]총자본회전률,5.419411e-03,2.789613e-03,1.781359e-03,1.260533e-03,9.483412e-04
[제조]비유동자산회전률,1.821720e-03,1.709879e-03,1.558423e-03,1.417368e-03,1.293263e-03
[제조]운전자본회전률,1.546809e-03,1.201184e-03,9.796866e-04,8.266811e-04,7.147191e-04
[제조]유동자산회전률,9.664330e-04,7.523543e-04,4.811121e-04,2.892155e-04,1.565565e-04
[제조]매출채권회전률,7.203055e-04,7.190718e-04,7.181074e-04,7.169244e-04,7.155560e-04
[제조]유동자산구성비율,6.964391e-04,6.344446e-04,5.993786e-04,5.766739e-04,5.606557e-04
[제조]총자본사업이익률,4.281350e-04,4.462992e-04,4.536193e-04,4.567984e-04,4.579161e-04
[제조]자기자본구성비율,3.568564e-04,3.701757e-04,3.745490e-04,3.759478e-04,3.760159e-04
[제조]1회전기간,1.486242e-04,7.881659e-05,5.380568e-05,4.084241e-05,3.291937e-05


# Lasso

In [47]:
lasso = Lasso(alpha = 10)
lasso_alphas =[0,0.1,1,10,100]

neg_mse_scores = cross_val_score(lasso, feature, target, scoring="neg_mean_squared_error", cv=5)
rmse_scores = np.sqrt(-1 * neg_mse_scores)
avg_rmse = np.mean(rmse_scores)
print(neg_mse_scores,rmse_scores,avg_rmse)

# lasso_alphas =[100,10000,10000]
for alpha in lasso_alphas :
    lasso = Lasso(alpha = alpha)
    neg_mse_scores = cross_val_score(lasso, feature, target, scoring="neg_mean_squared_error", cv=5)
    avg_rmse = np.mean(np.sqrt(-1*neg_mse_scores))
    print('alpha {0} 일 떄 5folds 의 평균 RMSE : {1:10f}' .format(alpha,avg_rmse))

# fig, axs = plt.subplots(figsize = (18,6), nrows = 1, ncols=5)

coeff_df = pd.DataFrame()

for pos, alpha in enumerate(lasso_alphas):
    lasso = Lasso(alpha = alpha)
    lasso.fit(feature,target)
    coeff = pd.Series(data=lasso.coef_, index=feature.columns)
    colname = 'alpha:' + str(alpha)
    coeff_df[colname] = coeff

    coeff = coeff.sort_values(ascending=False)

[-0.01123924 -0.01719316 -0.01216996 -0.01538518 -0.01266459] [0.10601528 0.13112268 0.11031756 0.124037   0.11253707] 0.1168059155628568


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.505e+01, tolerance: 1.212e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = 

alpha 0 일 떄 5folds 의 평균 RMSE :   0.113651
alpha 0.1 일 떄 5folds 의 평균 RMSE :   0.113772
alpha 1 일 떄 5folds 의 평균 RMSE :   0.115911
alpha 10 일 떄 5folds 의 평균 RMSE :   0.116806
alpha 100 일 떄 5folds 의 평균 RMSE :   0.116822


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_34068\2125100591.py:22: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso.fit(feature,target)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.618e+01, tolerance: 1.449e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_des

## Alpha 값 조정하면서 최적의 Alpha값 찾기

In [48]:
lasso_alphas =[0,0.1,1,10,100]

sort_column = 'alpha:'+str(lasso_alphas[0])
coeff_df.sort_values(by=sort_column, ascending=False)


,alpha:0,alpha:0.1,alpha:1,alpha:10,alpha:100
[제조]총자본회전률,3.144116e-02,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]순운전자본회전률,2.115825e-02,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]1회전기간,1.614939e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
[제조]운전자본회전률,1.999664e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
[제조]비유동자산회전률,9.711364e-04,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]유동자산구성비율,7.638032e-04,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]매출채권회전률,7.263543e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
[제조]총자본사업이익률,3.477668e-04,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]자기자본구성비율,2.814893e-04,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]부채비율,1.728272e-04,8.067027e-05,5.189188e-05,0.000000e+00,0.000000e+00


In [50]:

# a = coeff_df[coeff_df["alpha:0.25"]<-0.0000000001] 
# b = coeff_df[coeff_df["alpha:0.25"]>0.00000001]

# a.info()

In [ ]:
# feature = pd.concat([a,b])
# feature = feature["alpha:0.25"]
# feature = pd.DataFrame(feature)
# feature

,alpha:0.25
[제조]비유동자산증가율,-1.101468e-05
[제조]재고자산증가율,-1.642654e-05
[제조]매출액순이익률,-2.140773e-04
[제조]자기자본순이익률,-1.477270e-04
[제조]1주당매출액,-6.825699e-08
[제조]유동비율,-5.829955e-06
[제조]자기자본배율,-2.152229e-06
[제조]부가가치,-1.208220e-07
[제조]설비투자효율,-1.393240e-06
[제조]기계투자효율,-4.364865e-09


# ElasticNet

In [20]:
from sklearn.linear_model import ElasticNet
elasticNet = ElasticNet(alpha = 10)
alphas =[0.07,0.1,0.5,1,3]


# fig, axs = plt.subplots(figsize = (18,6), nrows = 1, ncols=5)

coeff_df = pd.DataFrame()

for pos, alpha in enumerate(alphas):
    
    elasticNet = ElasticNet(alpha = alpha, l1_ratio=0.07)
    elasticNet.fit(feature,target)
    coeff = pd.Series(data=elasticNet.coef_, index=feature.columns)
    colname = 'alpha:' + str(alpha)
    coeff_df[colname] = coeff

    coeff = coeff.sort_values(ascending=False)

## Alpha 값 조정하면서 최적의 Alpha값 찾기

In [49]:
elasticNet_alphas =[0,0.1,1,10,100]

sort_column = 'alpha:'+str(elasticNet_alphas[0])
coeff_df.sort_values(by=sort_column, ascending=False)


,alpha:0,alpha:0.1,alpha:1,alpha:10,alpha:100
[제조]총자본회전률,3.144116e-02,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]순운전자본회전률,2.115825e-02,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]1회전기간,1.614939e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
[제조]운전자본회전률,1.999664e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
[제조]비유동자산회전률,9.711364e-04,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]유동자산구성비율,7.638032e-04,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]매출채권회전률,7.263543e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
[제조]총자본사업이익률,3.477668e-04,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]자기자본구성비율,2.814893e-04,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
[제조]부채비율,1.728272e-04,8.067027e-05,5.189188e-05,0.000000e+00,0.000000e+00


In [ ]:
# ridge_alphas =[0,0.1,0.2,0.3,0.4,0.5]
# sort_column = 'alpha:'+str(ridge_alphas)
# coeff_df.sort_values(by=sort_column, ascending=False)

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np              
# from sklearn.metrics import mean_squared_error

# plt.scatter(Y_train,Y_pred_train, label = '(실제값, 예측값)', alpha=.5)
# plt.plot(np.linspace(0,20,200), np.linspace(0,50,200), color = 'green', label = "45°(실제값 = 예측값)")
# plt.xlabel("실제 Price: $Y_i$")
# plt.ylabel("예측 Price : $\hat{Y}_i$")
# plt.legend()
# plt.title("예측된 Price of train set vs 실제 Price ($Y_i$ vs $\hat{Y}_i$), " + f"alpha={alpha_value}")
# plt.text(y=0,x=35, s=f'MSE 값 : {mean_squared_error(Y_train,Y_pred_train) : .4f}')
# plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# from sklearn.metrics import mean_squared_error

# plt.scatter(Y_test,Y_pred, label = '(실제값, 예측값)', alpha=1)
# plt.plot(np.linspace(0,50,200), np.linspace(0,50,200), color = 'green', label = "45°(실제값 = 예측값)")
# plt.xlabel("실제 Price: $Y_i$")
# plt.ylabel("예측 Price: $\hat{Y}_i$")
# plt.legend()
# plt.title("예측 Price of test set vs 실제 Price ($Y_i$ vs $\hat{Y}_i$), " + f"alpha={alpha_value}")
# plt.text(y=0,x=35, s=f'MSE 값 : {mean_squared_error(Y_test,Y_pred) : .4f}')
# plt.show()

In [ ]:
# # 잔차 분석
# resid = Y_pred_train - Y_train

# plt.scatter(Y_pred_train, resid, c='b')
# plt.hlines(y=0, xmin= -10, xmax=50)
# plt.title('Residual plot')
# plt.show()